<a href="https://colab.research.google.com/github/kimathrareddy/Motorcycle-Simulation/blob/main/BMEG_230_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Hello

Welcom to Google Colab a cloud based computing platform running Python.

Before you do anything at all please make sure you have :
1. Have google account. If you dont make one just to access their cloud software.

2. The first time you ever open this I want you to save a local copy of it on your own google drive. TO do this go to FILE->SAVE A COPY IN DRIVE. This will be your own local version of this cloud based python code. You will from then on forward only ever use that version of the code. You can find it in your google drive, and save it somewhere you know where it is.

3. Once you have your own name it to something that you like. For example "CodeToRunOP"

4. Ths script will take about ~30 min to run. Therefore you need to make sure that once it starts running your computer does not fall asleep. I use an app called amphetamine on my MacBook that sets the computer to not fall asleep. You can also change the propoerties in your settings. Doesnt matter the approach.

5.  BEFORE RUNNING SCRIPTS PLEASE GO TO "RUNTIME-> FACTORY RESET RUNTIME" AND "EDIT->CLEAR ALL OUPPUTS"

6. CRITICAL Next Make sure when you go to "RunTime->Change runtime type " this is set to GPU. It will not work if you do not set this to GPU (do this everytime you comde back to the code after leaving it for a day).

DO NOT RUN THIS CODE BUT MAKE A COPY


**HOW DO I RUN THIS CODE ??**
After you have read all of the steps above (and completed them). You are ready to work. First drag in your video that you want to anlayze to the left hand side. Click the little folder if you cant see anywhere to drag into. Next scroll down to the little arrow below  "Install OpenPose" here you can run the first set of the code (~this will run for about 20-30min).

Once it is finsihed you can run the code on your video. Scroll down to try on sample video. Run this and if it works you will get an output. Sae this output its your result. You can run this box for every video you have just change the name where it is indicated. Dont forget to download all files after finishing as they will disaapear.

# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given video.

Please read the [OpenPose license](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/LICENSE) before running this script.


##Install OpenPose
Just run the code below but do not change anything :)

In [ ]:
#@title
import os
OPENPOSE_PATH="./openpose/"
HOME_PATH='./'

!echo $HOME_PATH

!pip install ffmpeg-python
from os.path import exists, join, basename, splitext

def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))


# see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
!tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
# clone openpose
!git clone -q --depth 1 $git_repo_url
# --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
!sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
!cd openpose && git submodule update --init --recursive --remote

! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh" -O models.zip && rm -rf /tmp/cookies.txt

#@title
# Workaround for server connection problem ("file DOWNLOAD HASH mismatch"),see, e.g.,
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/1602#issuecomment-641653411
!apt-get install unzip

# !wget -O models.zip  --no-check-certificate -r 'https://drive.google.com/uc?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download'
!unzip -o models.zip -d openpose

# use 'sed' to comment out the line in the OpenPose repo that downloads the model from the failed link
! sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.1.sh
! sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.3.sh
! sed -i 's/download_model("BODY_25"/# download_model("BODY_25"/g' ./openpose/CMakeLists.txt
! sed -i 's/78287B57CF85FA89C03F1393D368E5B7/# 78287B57CF85FA89C03F1393D368E5B7/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("body (COCO)"/# download_model("body (COCO)"/g' ./openpose/CMakeLists.txt
! sed -i 's/5156d31f670511fce9b4e28b403f2939/# 5156d31f670511fce9b4e28b403f2939/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("body (MPI)"/# download_model("body (MPI)"/g' ./openpose/CMakeLists.txt
! sed -i 's/2ca0990c7562bd7ae03f3f54afa96e00/# 2ca0990c7562bd7ae03f3f54afa96e00/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("face"/# download_model("face"/g' ./openpose/CMakeLists.txt
! sed -i 's/e747180d728fa4e4418c465828384333/# e747180d728fa4e4418c465828384333/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("hand"/# download_model("hand"/g' ./openpose/CMakeLists.txt
! sed -i 's/a82cfc3fea7c62f159e11bd3674c1531/# a82cfc3fea7c62f159e11bd3674c1531/g' ./openpose/CMakeLists.txt

! ls -lha /usr/lib/x86_64-linux-gnu

!cd openpose && rm -rf build || true && mkdir build

cmake_file='/content/openpose/CMakeLists.txt'
!cd openpose && sed -i 's/-DBUILD_python=OFF/-DBUILD_python=ON/g' $cmake_file
!cd openpose && sed -i 's/-DBUILD_python_layer=OFF/-DBUILD_python_layer=ON/g' $cmake_file

!cd openpose && sed -i 's/option(BUILD_PYTHON "Build OpenPose python." OFF)/option(BUILD_PYTHON "OpenPose python." ON)\noption(BUILD_BIN_FOLDER "Copy 3rd-party DLL files." ON)/g' $cmake_file

# CUDA
!cd openpose && cd build && cmake .. -DUSE_CUDNN=OFF -DGENERATE_PYTHON_BINDINGS:BOOL="1" -DPYTHON_LIBRARY='/usr/lib/x86_64-linux-gnu/libpython3.10.so' && make -j`nproc`



In [ ]:
# MOVING FORWARD YOU CAN RUN THIS BOX AND THE NEXY BOX FOR ANY NUMBER OF VIDEOS YOU WANT each vidoe should take
#approx 20-60 sec to procress. just change the names below to anaylze videos of different names and create different outputs.

# IMPORTANT HERE YOU WILL CHANGE this to the name of your file (note your file should be .mp4)
# below is just an example my file is named ExampleVideo
NameInputFile = 'IMG_5111'
# IMPORTANT HERE YOU WILL CHANGE this to the name of your output file (note your file should be .mp4)
# below is just an example my file output name is named ExampleVideo_OUTPUT
NameOutputFile = 'Will_45'

## REMEBER to save your output files after you run to get the data onto your computer :)
# NOTE if you get an empty csv file as an output then check if you named your video correctly, that it is a mp4 file.

keypoint_names = ['Nose','Neck','RShoulder','RElbow','RWrist','LShoulder','LElbow','LWrist','MidHip','RHip','RKnee','RAnkle','LHip','LKnee','LAnkle','REye','LEye','REar','LEar','LBigToe','LSmallToe','LHeel','RBigToe','RSmallToe','RHeel']

!cd $OPENPOSE_PATH && rm ../openpose.avi
!cd $OPENPOSE_PATH && chmod -R 755 './build/'
!cd $OPENPOSE_PATH && ./build/examples/openpose/openpose.bin --video ../{NameInputFile+'.mp4'} --write_json ./output/ --display 0 --number_people_max 1  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi {NameOutputFile+'.mp4'}

import os
import json
import csv

input_dir = '/content/openpose/output'
output_file= '/content/' + NameOutputFile +'.csv'

# Open the output CSV file for writing
with open(output_file, 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)

  # Write the header row to the CSV file
  header = ['Frame']
  for name in keypoint_names:
      for suffix in ['_x', '_y', '_conf']:
          header.append(name + suffix)
  writer.writerow(header)

  try:
      # Iterate over all JSON files in the input directory
      for filename in sorted(os.listdir(input_dir)):
          filepath = os.path.join(input_dir, filename)

          # Skip non-JSON files
          if not filename.endswith('.json'):
              continue

          print('Processing file {}'.format(filename))

          # Load the JSON data from the file
          with open(filepath, 'r') as f:
              data = json.load(f)

          # Skip files with no pose keypoints
          if len(data['people']) == 0:
              print('File {} does not contain any keypoints'.format(filename))
              continue

          # Extract the pose keypoints from the JSON data
          keypoints = data['people'][0]['pose_keypoints_2d']
          if keypoints is None:
              print('File {} does not contain any keypoints'.format(filename))
              continue

          # Write the keypoints to the CSV file
          row = [filename]
          for i in range(len(keypoint_names)):
              x = keypoints[i*3]
              y = keypoints[i*3 + 1]
              conf = keypoints[i*3 + 2]
              row.extend([x, y, conf])
          writer.writerow(row)

  except FileNotFoundError:
      print(f"Directory {input_dir} does not exist. Skipping...")




You can  visualize the result:

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video(NameOutputFile+'.mp4', width=960, height=720)

# New Section